<a href="https://colab.research.google.com/github/ellyonmagma/Mappia_Example/blob/main/C%C3%B3pia_de_tarefa_aula_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crie um programa no Colab que que execute as seguintes ações:

*   O usuário digita um nome de bairro, ele mostra a área deste bairro
em metros quadrados e o nome dos bairros adjacentes. Repita o
exercício com o Combobox.
*  O usuário digita um par de coordenadas, o programa diz a que
bairro ele pertence (nome do bairro). Mostre o resultado no mapa.
*   O usuário escolhe um bairro, aparece ele no mapa com os bairros
adjacentes. (Folium)
*   O usuário escolhe um bairro, aparecem as escolas dentro deste
bairro.

Execício 1

In [ ]:
# Preparando o ambiente no Google Drive:

#importando a biblioteca
from google.colab import drive

# Isso irá pedir sua autorização
drive.mount('/content/drive',force_remount=True)

# Agora, seu Drive estará disponível em: /content/drive/My Drive

Mounted at /content/drive


In [ ]:
# prompt: use geopandas para ler dois arquivos .shp do meu drive

import geopandas as gpd
import pandas as pd

# Caminhos para os arquivos .shp no seu Google Drive
bairros_shp = '/content/drive/My Drive/desenvolvimento de apps/aula02/DIVISA_DE_BAIRROS_SIRGAS/DIVISA_DE_BAIRROS.shp'  # Substitua pelo caminho correto
escolas_shp = '/content/drive/My Drive/desenvolvimento de apps/aula02/ESCOLA_MUNICIPAL_SIRGAS/ESCOLA_MUNICIPAL.shp'  # Substitua pelo caminho correto


bairros = gpd.read_file(bairros_shp)
escolas = gpd.read_file(escolas_shp)



In [ ]:
escolas.head()
bairros.head()

,OBJECTID,CODIGO,TIPO,NOME,FONTE,CD_REGIONA,NM_REGIONA,SHAPE_AREA,SHAPE_LEN,geometry
0,111,29.0,DIVISA DE BAIRROS,SEMINÁRIO,Decreto Municipal 774 / 1975,7.0,REGIONAL PORTÃO,2.123845e+06,7068.306546,"POLYGON ((671332.905 7184743.04, 671376.041 71..."
1,112,23.0,DIVISA DE BAIRROS,GUABIROTUBA,Decreto Municipal 774 / 1975,3.0,REGIONAL CAJURU,2.654431e+06,6576.157173,"POLYGON ((676773.573 7181043.613, 676773.533 7..."
2,113,27.0,DIVISA DE BAIRROS,PORTÃO,Decreto Municipal 774 / 1975,7.0,REGIONAL PORTÃO,5.858496e+06,10839.074473,"POLYGON ((671803.064 7182533.926, 671788.777 7..."
3,115,45.0,DIVISA DE BAIRROS,MOSSUNGUÊ,Decreto Municipal 774 / 1975,5.0,REGIONAL SANTA FELICIDADE,3.365441e+06,9125.525056,"POLYGON ((669136.262 7186337.442, 669520.266 7..."
4,116,24.0,DIVISA DE BAIRROS,PRADO VELHO,Decreto Municipal 774 / 1975,1.0,REGIONAL MATRIZ,2.434125e+06,7089.468068,"POLYGON ((676031.648 7184686.502, 676114.557 7..."


In [ ]:
mun = input("Digite o nome de um bairro: ")

areasaida = (bairros[bairros['NOME'] == mun].iloc[0].geometry.area)/10000

print('Tamanho do bairro em hectares: ',round(areasaida, 3) )


Digite o nome de um bairro: HAUER
Tamanho do bairro em hectares:  399.004


In [ ]:
from ipywidgets import Dropdown
from IPython.display import display, clear_output

# 2. Criar Dropdown com os municípios
comboMun = Dropdown(options=bairros.NOME.to_list(), description='Bairros')

# 3. Definir função de callback
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(comboMun)

        # Executa consulta com o código selecionado
        codigo = comboMun.value
        areasaida = (bairros[bairros['NOME'] == codigo].iloc[0].geometry.area)/10000

        print(f"\nÁrea em hectares do bairro selecionado: {areasaida:.2f}")

# 4. Associar a função ao Dropdown
comboMun.observe(on_change)

# 5. Mostrar o Dropdown na interface
display(comboMun)

Dropdown(description='Bairros', index=1, options=('SEMINÁRIO', 'GUABIROTUBA', 'PORTÃO', 'MOSSUNGUÊ', 'PRADO VE…


Área em hectares do bairro selecionado: 265.44


Exercicio 3

In [ ]:
import shapely.geometry
import pyproj

print('Insira um par de coordenadas:')
lat = float(input('Latitude: '))
lon = float(input('Longitude: '))

# Cria o ponto
ponto = shapely.geometry.Point(lon, lat)

# Definir os sistemas de coordenadas
origem_crs = 'EPSG:4674'
destino_crs = 'EPSG:31982'

# Transformer com ordem consistente
transformer = pyproj.Transformer.from_crs(origem_crs, destino_crs, always_xy=True)

# Transforma as coordenadas
x, y = transformer.transform(ponto.x, ponto.y)

# Cria o ponto transformado
ponto_sirgas = shapely.geometry.Point(x, y)

# Verifica em qual bairro está
bairro = bairros[bairros.contains(ponto_sirgas)]

# Resultado
if not bairro.empty:
    print(f"\nPonto está no bairro: {bairro.iloc[0]['NOME']}")
else:
    print("\nPonto não está em nenhum bairro definido.")












Insira um par de coordenadas:
Latitude: -25.47731525776276
Longitude: -49.247715072218696

Ponto está no bairro: HAUER


"\ntry:\n  print(bairro.iloc[0].NOME)\nexcept:\n  print('ponto fora da Extenção')\n  -25.47731525776276 -49.247715072218696\n\n  "

In [ ]:
import geopandas as gpd
import folium
from shapely.geometry import Polygon
from ipywidgets import Dropdown, Output, VBox
from IPython.display import display, clear_output

# Converter para WGS84 (Folium usa EPSG:4326)
bairros_wgs84 = bairros.to_crs(epsg=4326)

# Criar Dropdown com nomes de bairros
comboBairro = Dropdown(options=sorted(bairros_wgs84['NOME'].unique()), description='Bairro:')

# Saída para o mapa
out = Output()

def mostrar_mapa(bairro_nome):
    # Limpa saída anterior
    out.clear_output()

    # Seleciona bairro escolhido
    bairro_sel = bairros_wgs84[bairros_wgs84['NOME'] == bairro_nome]

    # Seleciona os bairros adjacentes
    vizinhos = bairros_wgs84[bairros_wgs84.touches(bairro_sel.unary_union)]

    # Centroide para centralizar o mapa
    centro = bairro_sel.geometry.centroid.iloc[0]
    mapa = folium.Map(location=[centro.y, centro.x], zoom_start=15)

    # Adiciona bairro selecionado em azul
    folium.GeoJson(
        bairro_sel,
        name='Bairro Selecionado',
        style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.3},
        tooltip=folium.GeoJsonTooltip(fields=['NOME'], aliases=['Bairro'])
    ).add_to(mapa)

    # Adiciona bairros adjacentes em vermelho
    folium.GeoJson(
        vizinhos,
        name='Bairros Vizinhos',
        style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.2},
        tooltip=folium.GeoJsonTooltip(fields=['NOME'], aliases=['Vizinho'])
    ).add_to(mapa)

    # Camadas
    folium.LayerControl().add_to(mapa)

    with out:
        display(mapa)

# Callback do Dropdown
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        mostrar_mapa(change['new'])

# Conecta o Dropdown ao callback
comboBairro.observe(on_change)

# Interface final
display(VBox([comboBairro, out]))


<ipython-input-32-062e78e0b811>:24: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  vizinhos = bairros_wgs84[bairros_wgs84.touches(bairro_sel.unary_union)]
<ipython-input-32-062e78e0b811>:27: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centro = bairro_sel.geometry.centroid.iloc[0]


exercicio 4

In [ ]:
import geopandas as gpd
import folium
from ipywidgets import Dropdown, Output, VBox
from IPython.display import display, clear_output

# 1. Converte para WGS84
bairros_wgs84 = bairros.to_crs(epsg=4326)
escolas_wgs84 = escolas.to_crs(epsg=4326)

# 2. Cria dropdown
comboBairro = Dropdown(options=sorted(bairros_wgs84['NOME'].unique()), description='Bairro:')
out = Output()

# 3. Função para mostrar mapa
def mostrar_mapa_escolas(bairro_nome):
    out.clear_output()

    # Seleciona o bairro
    bairro_sel = bairros_wgs84[bairros_wgs84['NOME'] == bairro_nome]

    # Filtra escolas dentro do bairro
    escolas_no_bairro = escolas_wgs84[escolas_wgs84.within(bairro_sel.unary_union)]

    # Centroide do bairro para centralizar o mapa
    centro = bairro_sel.geometry.centroid.iloc[0]
    mapa = folium.Map(location=[centro.y, centro.x], zoom_start=15)

    # Adiciona o bairro no mapa
    folium.GeoJson(
        bairro_sel,
        name='Bairro Selecionado',
        style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.3},
        tooltip=folium.GeoJsonTooltip(fields=['NOME'], aliases=['Bairro'])
    ).add_to(mapa)

    # Adiciona as escolas como marcadores
    for _, escola in escolas_no_bairro.iterrows():
        nome = escola.get('NOME', 'Escola')
        coords = escola.geometry.coords[0]
        folium.Marker(
            location=[coords[1], coords[0]],
            popup=nome,
            icon=folium.Icon(color='green', icon='graduation-cap', prefix='fa')
        ).add_to(mapa)

    folium.LayerControl().add_to(mapa)

    with out:
        display(mapa)

# 4. Função de callback
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        mostrar_mapa_escolas(change['new'])

comboBairro.observe(on_change)

# 5. Exibe a interface
display(VBox([comboBairro, out]))


<ipython-input-33-2982d386ad82>:22: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  escolas_no_bairro = escolas_wgs84[escolas_wgs84.within(bairro_sel.unary_union)]
<ipython-input-33-2982d386ad82>:25: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centro = bairro_sel.geometry.centroid.iloc[0]
